In [1]:
import pandas as pd 
import numpy as np

import dask.dataframe as dd
from dask_ml.preprocessing import DummyEncoder

import gc
import pprint

In [9]:
sess = dd.read_csv("D:/_DOCUMENTS/UBC Varisty Challenge/ubc_vc_2019_sessions.csv")
users = dd.read_csv("D:/_DOCUMENTS/UBC Varisty Challenge/ubc_vc_2019_users.csv")

In [10]:
sess.hc_store_visits = pd.to_numeric(sess.hc_store_visits)
sess.tutorial_completion = sess.tutorial_completion.astype('bool')
sess.sc_spent = pd.to_numeric(sess.sc_spent.fillna(0))

sess = sess.drop(['Unnamed: 0', 'session_id'], axis=1)
users = users.drop('Unnamed: 0', axis=1)
sess.columns

Index(['uid', 'connectivity', 'platform', 'in_alliance', 'session_start_time',
       'end_time', 'install_session', 'hc_store_visits', 'tutorial_completion',
       'tutorial_complete_timestamp', 'authentication_types', 'sc_spent',
       'sc_earned', 'xp_earned', 'ava_fights', 'ava_wins', 'ave_fights',
       'ave_wins', 'other_fights', 'other_wins', 'pve_fights', 'pve_wins',
       'pvp_fights', 'pvp_wins', 'total_fights', 'total_fight_time',
       'transactions', 'spend', 'level'],
      dtype='object')

In [19]:
sess.head(3)

,uid,connectivity,platform,in_alliance,session_start_time,end_time,install_session,hc_store_visits,tutorial_completion,tutorial_complete_timestamp,...,other_wins,pve_fights,pve_wins,pvp_fights,pvp_wins,total_fights,total_fight_time,transactions,spend,level
0,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,ON_WIFI,ios,True,2018-02-25 10:03:44+00:00,2018-02-25 11:21:19+00:00,False,0,False,NaN,...,0.0,71.0,67.0,0.0,0.0,71.0,1804.0,0,NaN,27.0
1,5M+BWt53rH5e9bFlDca5ho95CU+diI+A7lgstgMXchA=,ON_WIFI,ios,False,2018-02-28 10:21:28+00:00,2018-02-28 11:28:20+00:00,False,0,False,NaN,...,0.0,35.0,26.0,0.0,0.0,35.0,1738.0,0,NaN,27.0
2,qSOL52erIrvrHWneB5+vKRhFgNjCrizD0l822AwSDUU=,ON_WIFI,android,True,2018-02-23 14:40:50+00:00,2018-02-23 16:54:46+00:00,False,0,False,NaN,...,0.0,75.0,54.0,0.0,0.0,75.0,2187.0,0,NaN,27.0


In [4]:
#verify all users are unique
print(len(users))
print(users.uid.nunique().compute())
print(len(sess))
print(sess.uid.nunique().compute())

296248
296248


D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


3718080


D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


296248


In [11]:
#connectivity is a boolean var (It can only be wifi or mobile), so here I basically encoded Wifi as 1 (and mobile = 0)
#with the DummyEncoder to replace "connectivity". Same was done for platform
de1 = DummyEncoder(columns=['connectivity'], drop_first=True)
de2 = DummyEncoder(columns=['platform'], drop_first=True)

sess.connectivity = sess.connectivity.astype('category').cat.as_known()
sess.platform = sess.platform.astype('category').cat.as_known()
    
sess1 = de1.fit_transform(sess)
sess1 = de2.fit_transform(sess1)

D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


In [13]:
#Let's create a new feature to determine how long each session is, based on the start/end times
sess1.end_time = dd.to_datetime(sess1.end_time, utc=True)
sess1.session_start_time = dd.to_datetime(sess1.session_start_time, utc=True)
sess1['session_length'] = (sess1.end_time - sess1.session_start_time).astype('timedelta64[s]') #take difference and convert to seconds 
print(sess1['session_length'].head(3))

#it would also be interesting to see what percentage of time they spend "fighting". Here we're assuming fight time is in seconds
sess1['fight_ratio'] = sess1.total_fight_time/sess1.session_length
print(sess1.fight_ratio.head(3))

0    4655.0
1    4012.0
2    8036.0
Name: session_length, dtype: float64
0    0.38754
1    0.43320
2    0.27215
Name: fight_ratio, dtype: float64


In [20]:
#create new feature, amount spent per transaction
sess1['spent_per_transaction'] = sess1.spend/sess1.transactions if [sess.transactions>0] else 0

#some summary stats here:
print('')
print('Summary stats:')

print(sess1[sess1.tutorial_completion==1].session_length.mean().compute(), '= mean time to complete the tutorial (s)')

print((sess1.transactions).mean().compute(), '            = avg # of transactions per person:')

print(len(sess1[sess1.transactions>0])/len(sess1), '             = % of people that make a transaction')

print((sess1[sess1.transactions>0].spent_per_transaction).mean().compute(), '= avg amt spent per transaction')


Summary stats:
2002.2322408891384 = mean time to complete the tutorial (s)


D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


0.017339594629486185             = avg # of transactions per person:


D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


0.012122385747482572              = % of people that make a transaction
148.96292725893005 = avg amt spent per transaction


In [10]:
#see if there's any variation in spending between authentication types. We'll encode this as dummies later when users are joined with sessions
print(pd.get_dummies(sess.authentication_types, prefix='auth').columns)
sesst=pd.concat([sess, pd.get_dummies(sess.authentication_types, prefix='auth')], axis=1)
print(sesst.columns)
sesst['auth_google'] = sesst['auth_0']*sess.spend
statistics.mean(sesst['auth_0'])

#not significant. But I'll leave it here, maybe you'll get some other ideas to try

Index(['auth_0', 'auth_device', 'auth_gamecenter', 'auth_google',
       'auth_kabam'],
      dtype='object')
Index(['uid', 'platform', 'in_alliance', 'session_start_time', 'end_time',
       'install_session', 'hc_store_visits', 'tutorial_completion',
       'tutorial_complete_timestamp', 'authentication_types', 'sc_spent',
       'sc_earned', 'xp_earned', 'ava_fights', 'ava_wins', 'ave_fights',
       'ave_wins', 'other_fights', 'other_wins', 'pve_fights', 'pve_wins',
       'pvp_fights', 'pvp_wins', 'total_fights', 'total_fight_time',
       'transactions', 'spend', 'level', 'ON_WIFI', 'android', 'ios',
       'session_length', 'fight_ratio', 'spent_per_transaction', 'auth_0',
       'auth_device', 'auth_gamecenter', 'auth_google', 'auth_kabam'],
      dtype='object')


0.0002

In [28]:
users.head()

,uid,in_alliance,device_model,device_os,registered_email,num_friends,power,geo,language,level,total_logins,platform,install_time,first_spend_time,last_login_time,last_spend_time,lt_spend,tutorial_complete
0,iAVWIbMaJHm20cPrLxgABE3tbzMOlrFCthS+vk07h40=,True,SM-J730FM,Android OS 7.0,True,0,384.0,BY,ru,17.0,30,android,2018-02-09 20:08:51+00:00,NaN,2018-02-19 09:22:13+00:00,NaN,0.0,True
1,czL3c1UsK3BRtPbSckCby0ohFqyS7IbTChSUiD5vaGc=,False,Studio C HD,Android OS 6.0,False,0,117.0,TT,en,1.0,1,android,2018-02-02 21:39:16+00:00,NaN,2018-02-02 21:39:16+00:00,NaN,0.0,False
2,eyz3+noW7V1AiLQ1Ddkt3jtigmC9IRSHea/gyir2Yo0=,False,SM-J730F,Android OS 7.0,False,0,161.0,IL,it,9.0,24,android,2018-02-09 11:56:29+00:00,NaN,2018-02-15 11:51:33+00:00,NaN,0.0,True
3,7AbWa2/OIu4ac3+JA86d4J3RhWCsVbhuHtNjjER0scE=,False,SM-J730F,Android OS 7.0,False,0,571.0,BG,ru,25.0,68,android,2018-02-04 13:02:53+00:00,NaN,2018-02-21 15:19:17+00:00,NaN,0.0,True
4,XpyOIGDpHelCdpM96veWlu2qgGa4g5OwheuB0v8Tw5E=,False,SM-J500H,Android OS 6.0.1,True,0,118.0,HU,en,1.0,457,android,2018-02-24 15:16:53+00:00,NaN,2019-02-16 09:26:42+00:00,NaN,0.0,True


In [24]:
#how many people playing with each language
np.unique(users.language, return_counts=True)

(array(['ar', 'de', 'en', 'es', 'fr', 'id', 'it', 'ja', 'ko', 'nl', 'no',
        'pt', 'ru', 'th', 'tr', 'zh-CN', 'zh-TW'], dtype=object),
 array([  4617,   6316, 123607,  41355,   8481,   2502,   5307,   2392,
          1996,   1434,    116,  25819,  29422,   2875,  11167,  25914,
          2928], dtype=int64))

In [27]:
#total time played for users
users.install_time = dd.to_datetime(users.install_time, utc=True)
users.last_login_time = dd.to_datetime(users.last_login_time, utc=True)
users['played_for'] = (users.last_login_time - users.install_time).astype('timedelta64[s]') #take difference and convert to seconds 
print(users['played_for'].head(3))

0    825202.0
1         0.0
2    518104.0
Name: played_for, dtype: float64


In [31]:
#our top-spenders
users.sort_values(['lt_spend'], ascending=0)[:5]

,uid,in_alliance,device_model,device_os,registered_email,num_friends,power,geo,language,level,total_logins,platform,install_time,first_spend_time,last_login_time,last_spend_time,lt_spend,tutorial_complete,played_for
794,bPTU8yhARYbunhMGnHKVtSBUez4zWAb6kx1tc1Lvth8=,True,SM-G950F,Android OS 8.0.0,False,0,1147.0,CH,de,16.0,15,android,2018-02-25 02:18:11+00:00,2018-02-25 02:31:23+00:00,2018-04-01 01:36:06+00:00,2018-02-27 14:42:30+00:00,26462.8,True,3021475.0
231,NiPkCyVPr6N8AnHQ/YSRB3p/uqxTYhGJBnCuESkuOhc=,True,SM-N960F,Android OS 9,True,32,4840.0,RO,en,52.0,3795,android,2018-02-20 12:02:03+00:00,2018-02-26 11:38:15+00:00,2019-02-26 05:50:54+00:00,2019-02-25 10:32:33+00:00,24769.7,True,32032131.0
117,BipfNCixIYURe0Izg/rV8l8kghP3LlNbvt5gtiD5/JI=,True,Redmi Note 4,Android OS 7.0,True,0,1638.0,AE,ru,40.0,161,android,2018-02-03 13:31:34+00:00,2018-02-05 00:55:58+00:00,2018-03-01 21:35:15+00:00,2018-02-25 23:25:04+00:00,23717.0,True,2275421.0
697,05BWB9gvQYurVJC5NvPhE+YMF6NjsErJsgDhjZWNQXw=,True,SM-J510FN,Android OS 7.1.1,False,0,768.0,DZ,fr,6.0,4,android,2018-02-08 10:27:18+00:00,2018-02-08 10:52:21+00:00,2018-02-10 15:58:07+00:00,2018-02-09 13:45:06+00:00,17777.5,True,192649.0
409,NFud/1s6GOrSriEFiDO0Quv/lymJyZmV7MBesEeM9WU=,True,D2533,Android OS 5.0.2,True,1,1006.0,BY,ru,20.0,165,android,2018-02-03 22:26:27+00:00,2018-02-03 23:03:46+00:00,2018-12-31 00:48:17+00:00,2018-03-14 17:38:10+00:00,2985.7,True,28520510.0


In [28]:
print(users.uid.nunique().compute())
print(sess1.uid.nunique().compute())

296248


D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


296248


In [31]:
#match each session with an ID (jointype is inner so a session-less user will not appear)
joined = dd.merge(sess1,users,on='uid')
joined.head(5)

D:\Programs\Anaconda\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


,uid,in_alliance_x,session_start_time,end_time,install_session,hc_store_visits,tutorial_completion,tutorial_complete_timestamp,authentication_types,sc_spent,...,level_y,total_logins,platform,install_time,first_spend_time,last_login_time,last_spend_time,lt_spend,tutorial_complete,played_for
0,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-25 10:03:44+00:00,2018-02-25 11:21:19+00:00,False,0,False,NaN,kabam,200.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,NaN,2018-03-03 17:52:00+00:00,NaN,0.0,True,1212242.0
1,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-24 10:14:34+00:00,2018-02-24 11:19:50+00:00,False,1,False,NaN,kabam,15196.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,NaN,2018-03-03 17:52:00+00:00,NaN,0.0,True,1212242.0
2,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-25 13:52:57+00:00,2018-02-25 13:54:44+00:00,False,0,False,NaN,kabam,0.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,NaN,2018-03-03 17:52:00+00:00,NaN,0.0,True,1212242.0
3,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-25 09:39:45+00:00,2018-02-25 09:48:40+00:00,False,0,False,NaN,kabam,0.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,NaN,2018-03-03 17:52:00+00:00,NaN,0.0,True,1212242.0
4,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-24 14:57:40+00:00,2018-02-24 15:02:18+00:00,False,0,False,NaN,kabam,0.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,NaN,2018-03-03 17:52:00+00:00,NaN,0.0,True,1212242.0


In [32]:
print(joined.shape)
print(joined.columns)

(Delayed('int-9a4c646c-c842-4144-999c-236c21e835e5'), 50)
Index(['uid', 'in_alliance_x', 'session_start_time', 'end_time',
       'install_session', 'hc_store_visits', 'tutorial_completion',
       'tutorial_complete_timestamp', 'authentication_types', 'sc_spent',
       'sc_earned', 'xp_earned', 'ava_fights', 'ava_wins', 'ave_fights',
       'ave_wins', 'other_fights', 'other_wins', 'pve_fights', 'pve_wins',
       'pvp_fights', 'pvp_wins', 'total_fights', 'total_fight_time',
       'transactions', 'spend', 'level_x', 'connectivity_ON_WIFI',
       'platform_ios', 'session_length', 'fight_ratio',
       'spent_per_transaction', 'in_alliance_y', 'device_model', 'device_os',
       'registered_email', 'num_friends', 'power', 'geo', 'language',
       'level_y', 'total_logins', 'platform', 'install_time',
       'first_spend_time', 'last_login_time', 'last_spend_time', 'lt_spend',
       'tutorial_complete', 'played_for'],
      dtype='object')


In [19]:
for i in range(2):
    print('Collecting %d ...' % i)
    n = gc.collect()
    print('Unreachable objects:', n)
    print('Remaining Garbage:', )
    pprint.pprint(gc.garbage)
    print()

Unreachable objects: 3983
Remaining Garbage:
[]

Unreachable objects: 0
Remaining Garbage:
[]



In [34]:
joined.to_csv('D:/_DOCUMENTS/kabam_vc_joined-*.csv', compute=True)

['D:/_DOCUMENTS/kabam_vc_joined-00.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-01.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-02.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-03.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-04.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-05.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-06.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-07.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-08.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-09.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-10.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-11.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-12.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-13.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-14.csv',
 'D:/_DOCUMENTS/kabam_vc_joined-15.csv']